**Finding a string in files and removing it**

Removing #include<malloc.h> in all c files in a structure

In [18]:
%%bash
find . -name "*.c" | xargs sed -i bck "/#include<malloc\.h>/d"

In [20]:
%%bash 

cat ./isis/abs/allocate.cbck

#include<stdio.h>
#include<math.h>
#include<malloc.h>
#include<stdlib.h>


#include "types.h"

void allocate()
{
 int i,nc;


 i=deg_bg+2;

 nc=((i+1)*(i+2))/2;


 printf("nc: %i\n", nc);

 mat = (double **)malloc(nc*sizeof(double *));
 for(i=0;i<nc;i++)
  mat[i] = (double *)malloc(nc*sizeof(double));

 vec = (double *)malloc(nc*sizeof(double));
 lsvec = (double *)malloc(nc*sizeof(double));
 indx  = (int *)malloc(nc*sizeof(int));

}


**Quick introduction to chocolate bars and classes**

In [29]:
from astropy import units as u, constants as const

class SnickersBar(object):
    def __init__(self, w, h, l, weight, energy_density=2460 * u.kJ/ (100 * u.g)):
        self.w = u.Quantity(w, u.cm)
        self.h = u.Quantity(h, u.cm)
        self.l = u.Quantity(l, u.cm)
        self.weight = u.Quantity(weight, u.g)
        self.energy_density = u.Quantity(energy_density, u.kJ / u.g)
    
    def calculate_volume(self):
        return self.w * self.h * self.l
    
    @property
    def volume(self):
        return self.w * self.h * self.l
    
        

In [33]:
my_snickers_bar = SnickersBar(0.5, 0.5, 4, 0.01 * u.kg)
my_snickers_bar.w = 1 * u.cm

In [34]:
my_snickers_bar.volume

<Quantity 2.0 cm3>

Using cython

In [37]:
%load_ext Cython
import numpy as np

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [67]:
import numexpr as ne
x1, y1 = np.random.random((2, 1000000))
x2, y2 = np.random.random((2, 1000000))

distance = []

def calculate_distances(x1, y1, x2, y2):
    distances = []
    for i in xrange(len(x1)):
        distances.append(np.sqrt((x1[i] - x2[i])**2 + (y1[i] - y2[i]**2)))
    return distances

def numpy_calculate_distances(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1-y2)**2)


def ne_calculate_distances(x1, y1, x2, y2):
    return ne.evaluate('sqrt((x1 - x2)**2 + (y1-y2)**2)')

#%timeit calculate_distances(x1, y1, x2, y2)
%timeit ne_calculate_distances(x1, y1, x2, y2)

100 loops, best of 3: 3.03 ms per loop


In [64]:
%%cython -a

import numpy as np
cimport numpy as np
import cython

cdef extern from "math.h":
    cpdef double sqrt(double x)

@cython.boundscheck(False)
def cython_calculate_distances(double [:] x1, double [:] y1, double [:] x2, double [:] y2):
    distances = np.empty(len(x1))
    cdef double [:] distances_view = distances
    cdef int i
    cdef int len_x1=len(x1)
    for i in xrange(len_x1):
        distances_view[i] = sqrt((x1[i] - x2[i])**2 + (y1[i] - y2[i]**2))
    return distances

